In [13]:
import pandas as pd
import json, os

import tensorflow as tf
import numpy as np
import csv
from transformers import AutoTokenizer, TFAutoModel, TFAutoModelForSequenceClassification #, BertModel, BertTokenizer, TFBertForSequenceClassification
import matplotlib.pyplot as plt
import random
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow.keras.backend as K
from collections import OrderedDict
import time
from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score, \
roc_auc_score, confusion_matrix, classification_report
from sklearn.model_selection import StratifiedKFold
import random
import io

from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras import regularizers
from tensorflow.keras import layers
from tensorflow.keras.layers import LSTM, SimpleRNN
from tensorflow.keras.layers import GRU
from tensorflow.keras.layers import Masking
from tensorflow.keras.layers import Embedding, MaxPool1D
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow.keras import optimizers
from tensorflow.keras.constraints import max_norm
from tensorflow.keras.layers import Bidirectional, BatchNormalization
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.initializers import glorot_uniform, RandomUniform, lecun_uniform, Constant
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from collections import OrderedDict
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D, GlobalMaxPool1D
import tensorflow.keras.backend as K

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Dense, GlobalMaxPool1D

from sklearn.metrics import classification_report
from keras_preprocessing.text import tokenizer_from_json

import nltk
from nltk.tokenize import word_tokenize

import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC  
from sklearn.naive_bayes import GaussianNB
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler
from sklearn.utils import shuffle

In [14]:
seed = 123
np.random.seed(seed)
random.seed(seed)
tf.random.set_seed(seed)

In [15]:
embeddings = ["w2v", "ft", "bert", "codebert"]
embedding = embeddings[3]

Choose a project among Chrome and Linux or the full dataset

In [16]:
#data = pd.read_csv(os.path.join('..','data', 'full_data_reduced.csv'))
#data = pd.read_csv(os.path.join('..','data', 'chrome_data_reduced.csv'))
data = pd.read_csv(os.path.join('..','data', 'linux_data_reduced.csv'))

Shuffle the dataset before starting operating on it

In [17]:
data = data.sample(frac=1, random_state=seed).reset_index(drop=True)

In [18]:
print(data.head())

                                                func  vul  length
0  static inline void __ap_schedule_poll_timer(vo...    0      30
1  static int sctp_autobind(struct sock *sk)\n{\n...    0      36
2  static ssize_t ucma_init_qp_attr(struct ucma_f...    0      88
3  static void __blk_mq_requeue_request(struct re...    0      24
4  static void vhost_net_flush(struct vhost_net *...    0      37


In [19]:
np.max(data["length"])

237

In [20]:
word_counts = data["func"].apply(lambda x: len(x.split()))
max_length = word_counts.max()
print("Maximum number of words:", max_length)


Maximum number of words: 237


In [21]:
vc = data["vul"].value_counts()

print(vc)

print("Vulnerability Percentage: ", (vc[1] / vc[0])*100, '%')

0    43024
1     1439
Name: vul, dtype: int64
Vulnerability Percentage:  3.344644849386389 %


Train-val-test split

In [22]:
# split data into train and test and then train into train and val (90% train, 10% test and then 90% train and 10% val)
shuffle_seeders = [seed, 10, 15, 20, 25, 30, 35, 40, 45, 50]
shuffle_seeder = shuffle_seeders[0]

x_train_val, x_test, y_train_val, y_test = train_test_split(data["func"].tolist(), data["vul"].tolist(), stratify = data["vul"].tolist(), test_size=0.1, random_state=shuffle_seeder)

x_train, x_val, y_train, y_val = train_test_split(x_train_val, y_train_val, stratify = y_train_val, test_size=0.1, random_state=shuffle_seeder)


<b>Handling imbalanced data</b>

Class weighting

In [23]:
#total observations / (number of classes * observations in class)
class_weights = {0:len(data) / (len(vc) * vc[0]), 1:len(data) / (len(vc) * vc[1])}
#class_weights = {0:1, 1:1}
class_weights

{0: 0.516723224246932, 1: 15.449270326615705}

Under-sampling of the clean samples

In [24]:
# Apply under-sampling with the specified strategy
class_counts = pd.Series(y_train).value_counts()
print("Class distribution ", class_counts)
majority_class = class_counts.idxmax()
print("Majority class ", majority_class)
target_count = int(class_counts.iloc[0] / 2) 
print("Targeted number of majority class", target_count)

# under
sampling_strategy = {majority_class: target_count}        
rus = RandomUnderSampler(random_state=seed, sampling_strategy=sampling_strategy)
x_train_resampled, y_train_resampled = rus.fit_resample(x_train, y_train) 
print("Class distribution after under-sampling", pd.Series(y_train_resampled).value_counts())

Class distribution  0    34849
1     1165
dtype: int64
Majority class  0
Targeted number of majority class 17424


ValueError: Expected 2D array, got 1D array instead:
array=['syscall_define6(mbind, unsigned long, start, unsigned long, len,\n\t\tunsigned long, mode, const unsigned long __user *, nmask,\n\t\tunsigned long, maxnode, unsigned, flags)\n{\n\tnodemask_t nodes;\n\tint err;\n\tunsigned short mode_flags;\n\tmode_flags = mode & mpol_mode_flags;\n\tmode &= ~mpol_mode_flags;\n\tif (mode >= mpol_max)\n\t\treturn -einval;\n\tif ((mode_flags & mpol_f_static_nodes) &&\n\t    (mode_flags & mpol_f_relative_nodes))\n\t\treturn -einval;\n\terr = get_nodes(&nodes, nmask, maxnode);\n\tif (err)\n\t\treturn err;\n\treturn do_mbind(start, len, mode, mode_flags, &nodes, flags);\n}\n'
 'int sk_filter(struct sock *sk, struct sk_buff *skb)\n{\n\tint err;\n\tstruct sk_filter *filter;\n\t/*\n\t * if the skb was allocated from pfmemalloc reserves, only\n\t * allow sock_memalloc sockets to use it as this socket is\n\t * helping free memory\n\t */\n\tif (skb_pfmemalloc(skb) && !sock_flag(sk, sock_memalloc))\n\t\tre

Random Over-sampling of the vulnerable samples

In [ ]:
# # Apply random over-sampling with the specified strategy
# class_counts = pd.Series(y_train_resampled).value_counts()
# print("Class distribution ", class_counts)
# minority_class = class_counts.idxmin()
# print("Minority class ", minority_class)
# target_count = class_counts.iloc[0] #int(class_counts.iloc[1] * 2) 
# print("Targeted number of minority class", target_count)

# # over
# sampling_strategy = {minority_class: target_count}        
# ros = RandomOverSampler(random_state=seed, sampling_strategy=sampling_strategy)
# x_train_resampled, y_train_resampled = ros.fit_resample(x_train_resampled, y_train_resampled) 
# print("Class distribution after over-sampling", pd.Series(y_train_resampled).value_counts())

Synthetic Minority Over-sampling of the vulnerable samples

In [ ]:
# smote = SMOTE(random_state=seed, sampling_strategy='auto')  # 'auto' means it will resample to have the same number of samples as the majority class
# x_train_resampled, y_train_resampled = smote.fit_resample(x_train_resampled, y_train_resampled)

# # Class distribution after SMOTE
# class_counts_after_smote = pd.Series(y_train_resampled).value_counts()
# print("Class distribution after SMOTE", class_counts_after_smote)

Adaptive Synthetic Sampling with ADASYN - Over-sampling of the vulnerable samples

In [ ]:
# adasyn = ADASYN(random_state=seed, sampling_strategy='auto')  # 'auto' means it will resample to have the same number of samples as the majority class
# x_train_resampled, y_train_resampled = adasyn.fit_resample(x_train_resampled, y_train_resampled)

# # Class distribution after ADASYN
# class_counts_after_adasyn = pd.Series(y_train_resampled).value_counts()
# print("Class distribution after ADASYN", class_counts_after_adasyn)

In [ ]:
# Shuffle the resampled data while preserving the correspondence between features and labels
x_train_resampled, y_train_resampled = shuffle(x_train_resampled, y_train_resampled, random_state=seed)

# rename
X_train = x_train_resampled
Y_train = y_train_resampled

BERT pre-trained embeddings

In [ ]:
if embedding == "bert":
    model_variation = "bert-base-uncased" # "roberta-base"
    tokenizer = AutoTokenizer.from_pretrained(model_variation)
    bert = TFAutoModel.from_pretrained(model_variation)

In [ ]:
if embedding == "bert":
    bert_embeddings = bert.get_input_embeddings()
    embedding_matrix = bert_embeddings.weights[0].numpy()
    num_words = len(embedding_matrix)
    print(num_words)
    dim = len(embedding_matrix[0])
    print(dim)

In [ ]:
if embedding == "bert":
    sentences = x_train
    sequences = [tokenizer.encode(sente, truncation=True, add_special_tokens=False, return_tensors="tf").numpy() for sente in sentences] # Tokenize the complete sentences


In [ ]:
if embedding == "bert":
    lines_pad_x_train = []
    for seq in sequences:
        lines_pad_x_train.append(seq[0])

    lines_pad_x_train = pad_sequences(lines_pad_x_train, padding = 'post', maxlen = 512)
    lines_pad_x_train

In [ ]:
if embedding == "bert":
    sentences = x_val
    sequences = [tokenizer.encode(sente, truncation=True, add_special_tokens=False, return_tensors="tf").numpy() for sente in sentences] # Tokenize the complete sentences


In [ ]:
if embedding == "bert":
    lines_pad_x_val = []
    for seq in sequences:
        lines_pad_x_val.append(seq[0])

    lines_pad_x_val = pad_sequences(lines_pad_x_val, padding = 'post', maxlen = 512)
    lines_pad_x_val

CodeBERT pre-trained CPP embeddings

In [ ]:
if embedding == "codebert":
    model_variation = "microsoft/codebert-base-mlm" # "neulab/codebert-cpp"
    tokenizer = AutoTokenizer.from_pretrained(model_variation, do_lower_case=False) # , do_lower_case=True
    codebert = TFAutoModel.from_pretrained(model_variation) # , from_pt=True

In [ ]:
if embedding == "codebert":
    codebert_embeddings = codebert.get_input_embeddings()
    embedding_matrix = codebert_embeddings.weights[0].numpy()
    num_words = len(embedding_matrix)
    print(num_words)
    dim = len(embedding_matrix[0])
    print(dim)

In [ ]:
if embedding == "codebert":
    sentences = x_train
    sequences = [tokenizer(sente, return_tensors="tf", truncation=True, add_special_tokens=False) for sente in sentences]

In [ ]:
def padSequences(sequences, max_len):
    lines_pad = []
    for sequence in sequences:
        seq = sequence['input_ids'].numpy()[0]
        if len(seq) < max_len:
            for i in range(len(seq), max_len):
                seq = np.append(seq, 0)
        lines_pad.append(seq)
    return lines_pad


In [ ]:
def get_max_len(sequences):
    max_len = 0
    
    for seq in sequences:
        if len(seq['input_ids'].numpy()[0]) > max_len:
            max_len = len(seq['input_ids'].numpy()[0])
    
    return max_len

In [ ]:
max_len = get_max_len(sequences)
print(max_len)

In [ ]:
if embedding == "codebert":
    lines_pad_x_train = padSequences(sequences, max_len)
    lines_pad_x_train = [arr.tolist() for arr in lines_pad_x_train]
    lines_pad_x_train = np.array(lines_pad_x_train)

In [ ]:
if embedding == "codebert":
    sentences = x_val
    sequences = [tokenizer(sente, return_tensors="tf", truncation=True, add_special_tokens=False) for sente in sentences]

In [ ]:
if embedding == "codebert":
    lines_pad_x_val = padSequences(sequences, max_len)
    lines_pad_x_val = [arr.tolist() for arr in lines_pad_x_val]
    lines_pad_x_val = np.array(lines_pad_x_val)

Functions for Deep Learnig

In [ ]:
# Evaluation functions
def recall_metric(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = (true_positives + K.epsilon()) / (possible_positives + K.epsilon())
        return recall

def precision_metric(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = (true_positives + K.epsilon()) / (predicted_positives + K.epsilon())
        return precision

def f1_metric(y_true, y_pred):

    prec = precision_metric(y_true, y_pred)
    rec = recall_metric(y_true, y_pred)
    f1 = 2*((prec*rec)/(prec+rec+K.epsilon()))
    return f1

def f2_metric(y_true, y_pred):

    prec = precision_metric(y_true, y_pred)
    rec = recall_metric(y_true, y_pred)
    f2 = 5*((prec*rec)/(4*prec+rec+K.epsilon()))
    return f2

In [ ]:
# Deep Learning Models - Classifiers
def buildLstm(max_len, top_words, dim, seed, embedding_matrix):
    model=Sequential()
    #model.add(Embedding(input_dim=top_words+1, output_dim=dim, input_length=None, mask_zero=True))
    model.add(Embedding(input_dim=top_words, output_dim=dim, input_length=None, weights=[embedding_matrix], mask_zero=True, trainable=False))
    #model.add(SimpleRNN(300, dropout=0.3, stateful=False))
    model.add(LSTM(100, dropout=0.2, return_sequences=True, stateful=False))
    model.add(LSTM(50, dropout=0.1, stateful=False))
    #model.add(Bidirectional(LSTM(300, dropout=0.3, stateful=False)))
    #model.add(GRU(300, dropout=0.3, stateful=False))
    model.add(Activation('relu')) #dropout=0.2, recurrent_dropout=0.2, kernel_constraint=max_norm(3), bias_constraint=max_norm(3)
    model.add(BatchNormalization(momentum=0.0))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[f1_metric])  
    return model

def buildCnn(max_len, top_words, dim, seed, embedding_matrix):
    cnn_model = Sequential()
    cnn_model.add(Embedding(top_words, dim, input_length=None, weights=[embedding_matrix], mask_zero=True, trainable=False))
    cnn_model.add(Conv1D(filters = 128, kernel_size = 5, activation = 'relu'))
    '''cnn_model.add(MaxPooling1D(pool_size = 5))
    cnn_model.add(Conv1D(filters = 128, kernel_size = 5, activation = 'relu'))
    cnn_model.add(MaxPooling1D(pool_size = 5))
    cnn_model.add(Conv1D(filters = 128, kernel_size = 5, activation = 'relu'))'''
    cnn_model.add(GlobalMaxPool1D())
    #cnn_model.add(Dense(units = 128, activation = 'relu'))
    cnn_model.add(Dense(units = 1, activation = 'sigmoid'))
    cnn_model.compile(loss = "binary_crossentropy", optimizer = "adam", metrics=[f1_metric])
    return cnn_model


Training

In [ ]:
y_train = np.array(y_train)
y_val = np.array(y_val)

In [ ]:
nb_epoch = 100
BS = 64
print("Training...")
milli_sec1 = int(round(time.time() * 1000))

userModel = "lstm"

if userModel == "cnn":
    myModel = buildCnn(max_len, num_words, dim, seed, embedding_matrix) 
elif userModel == "lstm":
    myModel = buildLstm(max_len, num_words, dim, seed, embedding_matrix)
    
print("model summary\m", myModel.summary())
csv_logger = CSVLogger('log.csv', append=True, separator=',')
es = EarlyStopping(monitor='val_f1_metric', mode='max', verbose=1, patience=10)
mc = ModelCheckpoint('best_model.h5', monitor='val_f1_metric', mode='max', verbose=1, save_best_only=True)

history = myModel.fit(lines_pad_x_train, y_train, validation_data=(lines_pad_x_val, y_val), epochs = nb_epoch, batch_size = BS, shuffle=True, verbose=1, callbacks=[csv_logger,es,mc], class_weight=class_weights)

#load best model
#model = load_model('best_model.h5')
myModel.load_weights("best_model.h5")

scores = myModel.evaluate(lines_pad_x_val, y_val, verbose=0)
#predictions = myModel.predict_classes(X_test, verbose=0)
predScores = myModel.predict(lines_pad_x_val)
predictions = (predScores > 0.5).astype("int32")

accuracy=accuracy_score(y_val, predictions)
precision=precision_score(y_val, predictions)
recall=recall_score(y_val, predictions)
f1=f1_score(y_val, predictions)
roc_auc=roc_auc_score(y_val, predictions)
f2=5*precision*recall / (4*precision+recall)

cm = confusion_matrix(y_val, predictions, labels=[0, 1])
#print(cm)
sns.heatmap(cm, annot=True)
tn, fp, fn, tp = cm.ravel()

print("TP=",tp)
print("TN=",tn)
print("FP=",fp)
print("FN=",fn)

acc = ((tp+tn)/(tp+tn+fp+fn))

print("Accuracy:%.2f%%"%(acc*100))
print("Precision:%.2f%%"%(precision*100))
print("Recall:%.2f%%"%(recall*100))
print("F1 score:%.2f%%"%(f1*100))
print("Roc_Auc score:%.2f%%"%(roc_auc*100))
print("F2 score:%.2f%%"%(f2*100))
print(classification_report(y_val, predictions))

milli_sec2 = int(round(time.time() * 1000))
print("Cross Validation is completed after", milli_sec2-milli_sec1)